In [ ]:
#| default_exp preprocessing

# Preprocessing
> Utilities for processing data before training/analysis

In [ ]:
#| export
import warnings
from datetime import date, datetime
from typing import Union

import numpy as np
import pandas as pd

from utilsforecast.compat import DataFrame, pl, pl_DataFrame, pl_Series
from utilsforecast.processing import group_by, repeat
from utilsforecast.validation import _is_int_dtype

In [ ]:
#| hide
from nbdev import show_doc

from utilsforecast.data import generate_series

In [ ]:
#| exporti
def _determine_bound(bound, freq, times_by_id, agg) -> np.ndarray:
    if bound == 'per_serie':
        out = times_by_id[agg].values
    else:
        # the following return a scalar
        if bound == 'global':
            val = getattr(times_by_id[agg].values, agg)()
            if isinstance(freq, str):
                val = np.datetime64(val)
        else:
            if isinstance(freq, str):
                # this raises a nice error message if it isn't a valid datetime
                val = np.datetime64(bound)
            else:
                val = bound
        out = np.full(times_by_id.shape[0], val)
    if isinstance(freq, str):
        out = out.astype(f'datetime64[{freq}]')
    return out

In [ ]:
#| exporti
def _determine_bound_pl(
    bound: Union[str, int, date, datetime],
    times_by_id: pl_DataFrame,
    agg: str,
) -> pl_Series:
    if bound == 'per_serie':
        out = times_by_id[agg]
    else:
        if bound == 'global':
            val = getattr(times_by_id[agg], agg)()
        else:
            val = bound
        out = repeat(pl_Series([val]), times_by_id.shape[0])
    return out

In [ ]:
#| export
def fill_gaps(
    df: DataFrame,
    freq: Union[str, int],
    start: Union[str, int, date, datetime] = 'per_serie',
    end: Union[str, int, date, datetime] = 'global',
    id_col: str = 'unique_id',
    time_col: str = 'ds',
) -> pd.DataFrame:
    """Enforce start and end datetimes for dataframe.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input data
    freq : str or int
        Series' frequency
    start : str, int, date or datetime.
        Initial timestamp for the series.
            * 'per_serie' uses each serie's first timestamp
            * 'global' uses the first timestamp seen in the data
            * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1)
    end : str, int, date or datetime.
        Initial timestamp for the series.
            * 'per_serie' uses each serie's last timestamp
            * 'global' uses the last timestamp seen in the data
            * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1)
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestamp.

    Returns
    -------
    filled_df : pandas or polars DataFrame
        Dataframe with gaps filled.
    """
    if isinstance(df, pl_DataFrame):
        times_by_id = group_by(df, id_col).agg(
            pl.col(time_col).min().alias('min'),
            pl.col(time_col).max().alias('max'),
        ).sort(id_col)
        starts = _determine_bound_pl(start, times_by_id, 'min')
        ends = _determine_bound_pl(end, times_by_id, 'max')
        grid = pl_DataFrame({id_col: times_by_id[id_col]})
        if _is_int_dtype(starts):
            grid = grid.with_columns(
                pl.int_ranges(starts, ends + freq, step=freq, eager=True).alias(time_col)
            )
        else:
            if starts.dtype == pl.Date:
                ranges_fn = pl.date_ranges
            else:
                ranges_fn = pl.datetime_ranges
            grid = grid.with_columns(
                ranges_fn(
                    starts,
                    ends,
                    interval=freq,
                    eager=True,
                ).alias(time_col)
            )
        grid = grid.explode(time_col)
        return grid.join(df, on=[id_col, time_col], how='left')
    if isinstance(freq, str):
        offset = pd.tseries.frequencies.to_offset(freq)
        if isinstance(offset.base, pd.offsets.Minute):
            # minutes are represented as 'm' in numpy
            freq = 'm'
        elif isinstance(offset.base, pd.offsets.BusinessDay):
            if offset.n != 1:
                raise NotImplementedError('Multiple of a business day')
            freq = 'D'
        elif isinstance(offset.base, pd.offsets.Hour):
            # hours are represented as 'h' in numpy
            freq = 'h'
        if offset.n > 1:
            freq = freq.replace(str(offset.n), '')
        try:
            pd.Timedelta(offset)
        except ValueError:
            # irregular freq, try using first letter of abbreviation
            # such as MS = 'Month Start' -> 'M', YS = 'Year Start' -> 'Y'
            freq = freq[0]
        delta: Union[np.timedelta64, int] = np.timedelta64(offset.n, freq)
    else:
        delta = freq
    times_by_id = df.groupby(id_col, observed=True)[time_col].agg(['min', 'max'])
    starts = _determine_bound(start, freq, times_by_id, 'min')
    ends = _determine_bound(end, freq, times_by_id, 'max') + delta
    sizes = ((ends - starts) / delta).astype(np.int64)
    times = np.hstack(
        [
            np.arange(start, end, delta) for start, end in zip(starts, ends)
        ]
    )
    uids = np.repeat(times_by_id.index, sizes)    
    if isinstance(freq, str):
        if isinstance(offset.base, pd.offsets.BusinessDay):
            # data was generated daily, we need to keep only business days 
            bdays = np.is_busday(times)
            uids = uids[bdays]
            times = times[bdays]
        times = pd.Index(times.astype('datetime64[ns]', copy=False))
        first_time = np.datetime64(df.iloc[0][time_col])
        was_truncated = first_time != first_time.astype(f'datetime64[{freq}]')
        if was_truncated:
            times += offset.base
    idx = pd.MultiIndex.from_arrays([uids, times], names=[id_col, time_col])
    res = df.set_index([id_col, time_col]).reindex(idx).reset_index()
    extra_cols = df.columns.drop([id_col, time_col]).tolist()
    if extra_cols:
        check_col = extra_cols[0]
        if res[check_col].count() < df[check_col].count():
            warnings.warn(
                "Some values were lost during filling, "
                "please make sure that all your times meet the specified frequency.\n"
                "For example if you have 'W-TUE' as your frequency, "
                "make sure that all your times are actually Tuesdays."
            )
    return res

In [ ]:
show_doc(fill_gaps)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/preprocessing.py#L56){target="_blank" style="float:right; font-size:smaller"}

### fill_gaps

>      fill_gaps
>                 (df:Union[pandas.core.frame.DataFrame,polars.dataframe.frame.D
>                 ataFrame], freq:Union[str,int], start:Union[str,int,datetime.d
>                 ate,datetime.datetime]='per_serie',
>                 end:Union[str,int,datetime.date,datetime.datetime]='global',
>                 id_col:str='unique_id', time_col:str='ds')

Enforce start and end datetimes for dataframe.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Input data |
| freq | Union |  | Series' frequency |
| start | Union | per_serie | Initial timestamp for the series.<br>    * 'per_serie' uses each serie's first timestamp<br>    * 'global' uses the first timestamp seen in the data<br>    * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1) |
| end | Union | global | Initial timestamp for the series.<br>    * 'per_serie' uses each serie's last timestamp<br>    * 'global' uses the last timestamp seen in the data<br>    * Can also be a specific timestamp or integer, e.g. '2000-01-01', 2000 or datetime(2000, 1, 1) |
| id_col | str | unique_id | Column that identifies each serie. |
| time_col | str | ds | Column that identifies each timestamp. |
| **Returns** | **DataFrame** |  | **Dataframe with gaps filled.** |

In [ ]:
df = pd.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': pd.to_datetime(['2020', '2021', '2023', '2021', '2022']),
        'y': np.arange(5),
    }
)
df

,unique_id,ds,y
0,0,2020-01-01,0
1,0,2021-01-01,1
2,0,2023-01-01,2
3,1,2021-01-01,3
4,1,2022-01-01,4


The default functionality is taking the current starts and only extending the end date to be the same for all series.

In [ ]:
fill_gaps(
    df,
    freq='YS',
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,1,2021-01-01,3.0
5,1,2022-01-01,4.0
6,1,2023-01-01,NaN


We can also specify `end='per_serie'` to only fill possible gaps within each serie.

In [ ]:
fill_gaps(
    df,
    freq='YS',
    end='per_serie',
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,1,2021-01-01,3.0
5,1,2022-01-01,4.0


We can also specify an end date in the future.

In [ ]:
fill_gaps(
    df,
    freq='YS',
    end='2024',
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,0,2024-01-01,NaN
5,1,2021-01-01,3.0
6,1,2022-01-01,4.0
7,1,2023-01-01,NaN
8,1,2024-01-01,NaN


We can set all series to start at the same time.

In [ ]:
fill_gaps(
    df,
    freq='YS',
    start='global'
)

,unique_id,ds,y
0,0,2020-01-01,0.0
1,0,2021-01-01,1.0
2,0,2022-01-01,NaN
3,0,2023-01-01,2.0
4,1,2020-01-01,NaN
5,1,2021-01-01,3.0
6,1,2022-01-01,4.0
7,1,2023-01-01,NaN


We can also set a common start date for all series (which can be earlier than their current starts).

In [ ]:
fill_gaps(
    df,
    freq='YS',
    start='2019',
)

,unique_id,ds,y
0,0,2019-01-01,NaN
1,0,2020-01-01,0.0
2,0,2021-01-01,1.0
3,0,2022-01-01,NaN
4,0,2023-01-01,2.0
5,1,2019-01-01,NaN
6,1,2020-01-01,NaN
7,1,2021-01-01,3.0
8,1,2022-01-01,4.0
9,1,2023-01-01,NaN


In case the times are integers the frequency, start and end must also be integers.

In [ ]:
df = pd.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [2020, 2021, 2023, 2021, 2022],
        'y': np.arange(5),
    }
)
df

,unique_id,ds,y
0,0,2020,0
1,0,2021,1
2,0,2023,2
3,1,2021,3
4,1,2022,4


In [ ]:
fill_gaps(
    df,
    freq=1,
    start=2019,
    end=2024,
)

,unique_id,ds,y
0,0,2019,NaN
1,0,2020,0.0
2,0,2021,1.0
3,0,2022,NaN
4,0,2023,2.0
5,0,2024,NaN
6,1,2019,NaN
7,1,2020,NaN
8,1,2021,3.0
9,1,2022,4.0


The function also accepts polars dataframes

In [ ]:
#| polars
df = pl.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [
            datetime(2020, 1, 1), datetime(2022, 1, 1), datetime(2023, 1, 1),
            datetime(2021, 1, 1), datetime(2022, 1, 1)],
        'y': np.arange(5),
    }
)
df

unique_id,ds,y
i64,datetime[μs],i64
0,2020-01-01 00:00:00,0
0,2022-01-01 00:00:00,1
0,2023-01-01 00:00:00,2
1,2021-01-01 00:00:00,3
1,2022-01-01 00:00:00,4


In [ ]:
#| polars
fill_gaps(
    df,
    freq='1y',
    start=datetime(2019, 1, 1),
    end=datetime(2024, 1, 1),
)

unique_id,ds,y
i64,datetime[μs],i64
0,2019-01-01 00:00:00,null
0,2020-01-01 00:00:00,0
0,2021-01-01 00:00:00,null
0,2022-01-01 00:00:00,1
0,2023-01-01 00:00:00,2
0,2024-01-01 00:00:00,null
1,2019-01-01 00:00:00,null
1,2020-01-01 00:00:00,null
1,2021-01-01 00:00:00,3


In [ ]:
#| polars
df = pl.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [
            date(2020, 1, 1), date(2022, 1, 1), date(2023, 1, 1),
            date(2021, 1, 1), date(2022, 1, 1)],
        'y': np.arange(5),
    }
)
df

unique_id,ds,y
i64,date,i64
0,2020-01-01,0
0,2022-01-01,1
0,2023-01-01,2
1,2021-01-01,3
1,2022-01-01,4


In [ ]:
#| polars
fill_gaps(
    df,
    freq='1y',
    start=date(2020, 1, 1),
    end=date(2024, 1, 1),
)

unique_id,ds,y
i64,date,i64
0,2020-01-01,0
0,2021-01-01,null
0,2022-01-01,1
0,2023-01-01,2
0,2024-01-01,null
1,2020-01-01,null
1,2021-01-01,3
1,2022-01-01,4
1,2023-01-01,null


In [ ]:
#| polars
df = pl.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': [2020, 2021, 2023, 2021, 2022],
        'y': np.arange(5),
    }
)
df

unique_id,ds,y
i64,i64,i64
0,2020,0
0,2021,1
0,2023,2
1,2021,3
1,2022,4


In [ ]:
#| polars
fill_gaps(
    df,
    freq=1,
    start=2019,
    end=2024,
)

unique_id,ds,y
i64,i64,i64
0,2019,null
0,2020,0
0,2021,1
0,2022,null
0,2023,2
0,2024,null
1,2019,null
1,2020,null
1,2021,3


In [ ]:
#| hide
from itertools import product

from fastcore.test import test_warns

In [ ]:
#| hide
def check_fill(dates, freq, start, end, include_start, include_end):    
    min_idx = 1 if include_start else 2
    max_idx = dates.size if include_end else dates.size - 1
    date_idxs = np.hstack(
        [
            np.random.choice(
                np.arange(min_idx, max_idx),
                size=n_periods // 2,
                replace=False
            ) for _ in range(2)
        ]
    )
    data = pd.DataFrame(
        {
            'unique_id': np.repeat([1, 2], n_periods // 2),
            'ds': dates[date_idxs],
            'y': np.arange(n_periods, dtype=np.float64)
        }
    )
    filled = fill_gaps(data, freq, start=start, end=end)
    data_starts_ends = data.groupby('unique_id', observed=True)['ds'].agg(['min', 'max'])
    global_start = data_starts_ends['min'].min()
    global_end = data_starts_ends['max'].max()
    filled_starts_ends = filled.groupby('unique_id', observed=True)['ds'].agg(['min', 'max'])

    # inferred frequency is the expected
    first_serie = filled[filled['unique_id'] == 1]
    if isinstance(freq, str):
        inferred_freq = pd.infer_freq(first_serie['ds'])
        assert inferred_freq == pd.tseries.frequencies.to_offset(freq)
    else:
        assert all(first_serie['ds'].diff().value_counts().index == [freq])
    
    # fill keeps original data
    assert filled['y'].count() == n_periods
    # check starts
    if start == 'per_serie':
        pd.testing.assert_series_equal(
            data_starts_ends['min'], filled_starts_ends['min'],
        )        
    else: # global or specific
        min_dates = filled_starts_ends['min'].unique()
        assert min_dates.size == 1
        expected_start = global_start if start == 'global' else start
        assert min_dates[0] == expected_start

    # check ends
    if end == 'per_serie':
        pd.testing.assert_series_equal(
            data_starts_ends['max'], filled_starts_ends['max'],
        )        
    else: # global or specific
        max_dates = filled_starts_ends['max'].unique()
        assert max_dates.size == 1
        expected_end = global_end if end == 'global' else end
        assert max_dates[0] == expected_end

n_periods = 100
freqs = ['YE', 'YS', 'ME', 'MS', 'W', 'W-TUE', 'D', 's', 'ms', 1, 2, '20D', '30s', '2YE', '3YS', '30min', 'B', '1h']
try:
    pd.tseries.frequencies.to_offset('YE')
except ValueError:
    freqs = [f.replace('YE', 'Y').replace('ME', 'M').replace('h', 'H') for f in freqs if isinstance(f, str)]
for freq in freqs:
    if isinstance(freq, (pd.offsets.BaseOffset, str)):         
        dates = pd.date_range('1900-01-01', periods=n_periods, freq=freq)
        offset = pd.tseries.frequencies.to_offset(freq)
    else:
        dates = np.arange(0, freq * n_periods, freq, dtype=np.int64)
        offset = freq
    global_start = dates[0]
    global_end = dates[-1]
    starts = ['global', 'per_serie', global_start - offset]
    ends = ['global', 'per_serie', global_end + offset]
    include_starts = [True, False]
    include_ends = [True, False]
    iterable = product(starts, ends, include_starts, include_ends)
    for start, end, include_start, include_end in iterable:
        check_fill(dates, freq, start, end, include_start, include_end)

In [ ]:
#| hide
# last value doesn't meet frequency (year start)
dfx = pd.DataFrame(
    {
        'unique_id': [0, 0, 0, 1, 1],
        'ds': pd.to_datetime(['2020-01', '2021-01', '2023-01', '2021-01', '2022-02']),
        'y': np.arange(5),
    }
)
with warnings.catch_warnings(record=True) as w:
    fill_gaps(dfx, 'YS')
assert 'values were lost' in str(w[0].message)